In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from datetime import datetime
import mysql.connector 
import pymysql
import os

In [2]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="password",
  database="test",
  auth_plugin="mysql_native_password",
  use_unicode=True, 
  charset="utf8"
)

# TB SCREENING

In [3]:
data = pd.read_sql("select PatientID,DateOfBirth,Sex from tblpatients", con=mydb);

In [4]:
data.head()

,PatientID,DateOfBirth,Sex
0,00-0A-09-2013-A-00421,1988-08-20,Female
1,00-0A-09-2019-A-00630,1965-10-05,Female
2,00-0A-0C-2009-A-00335,1971-11-21,Female
3,00-0A-0C-2011-A-03012,1960-06-06,Male
4,00-0A-0D-2007-A-04805,1971-12-18,Male


In [5]:
data_va = pd.read_sql("select tblvisits.PatientID,tblpatients.DateOfBirth,tblpatients.Sex,tblvisits.VisitDate,tblvisits.VisitTypeCode,tblsetuptbstatus.TBStatusDescription,tblsetuparvstatuscodes.ARVStatusDescription from tblvisits JOIN tblpatients ON tblvisits.PatientID = tblpatients.PatientID JOIN tblsetuptbstatus ON tblvisits.TBStatusCode=tblsetuptbstatus.TBStatusCode JOIN tblsetuparvstatuscodes ON tblvisits.ARVStatusCode =tblsetuparvstatuscodes.ARVStatusCode ", con=mydb);

In [6]:
data_va.head()

,PatientID,DateOfBirth,Sex,VisitDate,VisitTypeCode,TBStatusDescription,ARVStatusDescription
0,01-02-62-2008-A-00159,1982-06-05,Male,2015-09-11,P,No signs (patient screened and TB not suspected),No ARV
1,01-02-62-2008-A-00159,1982-06-05,Male,2017-02-06,P,No signs (patient screened and TB not suspected),No ARV
2,02-01-62-2015-A-00359,1987-05-26,Female,2015-07-01,P,No signs (patient screened and TB not suspected),No ARV
3,02-01-62-2015-A-00359,1987-05-26,Female,2015-07-29,P,No signs (patient screened and TB not suspected),No ARV
4,02-01-62-2015-A-00360,2010-07-14,Male,2015-07-01,P,No signs (patient screened and TB not suspected),No ARV


In [7]:
med = pd.read_sql("select t.PatientID,t.VisitDate,t.DrugTypeID from tblmedication t where t.DrugTypeID = 'IPT' OR t.DrugTypeID = 'FLU' OR t.DrugTypeID = 'TPT'", con=mydb);

In [8]:
med.head()

,PatientID,VisitDate,DrugTypeID
0,02-01-62-2015-A-00359,2017-04-26,FLU
1,02-07-0B-2008-A-00390,2016-07-08,FLU
2,02-07-0B-2010-A-00087,2017-04-19,FLU
3,02-07-0B-2010-A-00140,2017-10-19,FLU
4,02-07-0B-2011-A-00178,2019-02-11,FLU


In [9]:
data_v = pd.merge(data_va, med, how='left', on=['PatientID', 'VisitDate'])

In [10]:
data_v.head()

,PatientID,DateOfBirth,Sex,VisitDate,VisitTypeCode,TBStatusDescription,ARVStatusDescription,DrugTypeID
0,01-02-62-2008-A-00159,1982-06-05,Male,2015-09-11,P,No signs (patient screened and TB not suspected),No ARV,NaN
1,01-02-62-2008-A-00159,1982-06-05,Male,2017-02-06,P,No signs (patient screened and TB not suspected),No ARV,NaN
2,02-01-62-2015-A-00359,1987-05-26,Female,2015-07-01,P,No signs (patient screened and TB not suspected),No ARV,NaN
3,02-01-62-2015-A-00359,1987-05-26,Female,2015-07-29,P,No signs (patient screened and TB not suspected),No ARV,NaN
4,02-01-62-2015-A-00360,2010-07-14,Male,2015-07-01,P,No signs (patient screened and TB not suspected),No ARV,NaN


In [11]:
fac=pd.read_sql("select c.regioncode,c.districtcode,c.facilitycode from tblconfig c",con=mydb)

In [12]:
facii=pd.read_sql("select * from facilities",con=mydb)

In [13]:
facii.head()

,FacilityID,ProvinceCode,DistrictCode,FacilityCode,FacilityName,FullFacilityCode,FullFacilityCodeDashes
0,1,0,A,01,Avondale Clinic,000A01,00-0A-01
1,2,0,A,02,Arcadia P.C.C.,000A02,00-0A-02
2,3,0,A,03,Bank Street P.C.C.,000A03,00-0A-03
3,4,0,A,05,Belvedere F.H.C.,000A05,00-0A-05
4,5,0,A,06,Borrowdale F.H.C.,000A06,00-0A-06


In [14]:
reg=pd.read_sql("select * from tblsetupregions",con=mydb)

In [15]:
dist=pd.read_sql("select * from tblsetupdistricts",con=mydb)

In [16]:
dist.head()

,RegionCode,DistrictCode,District
0,0,1,Harare Rural
1,0,A,Harare Urban
2,0,B,Chitungwisa Urban
3,0,C,Chitungwiza Hospital
4,0,D,Harare Central Hospital


In [17]:
#tblconfig.regioncode=tblsetupregions.RegionCode

In [18]:
n=data_v.shape[0]

In [19]:
## Facility
data_v['Facilitycode']=fac['facilitycode']
faility_name=data_v['Facilitycode']
faility_name[1:n] = faility_name[0]
data_v['Facilitycode']=faility_name
filename= faility_name[0]
## District
data_v['Districtcode']=fac['districtcode']
district_name=data_v['Districtcode']
district_name[1:n] = district_name[0]
data_v['Districtcode']=district_name
## Province
data_v['Provincedoce']=fac['regioncode']
province_name=data_v['Provincedoce']
province_name[1:n] = province_name[0]
data_v['Provincedoce']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [20]:
data_v.head()

,PatientID,DateOfBirth,Sex,VisitDate,VisitTypeCode,TBStatusDescription,ARVStatusDescription,DrugTypeID,Facilitycode,Districtcode,Provincedoce
0,01-02-62-2008-A-00159,1982-06-05,Male,2015-09-11,P,No signs (patient screened and TB not suspected),No ARV,NaN,0B,7,2.0
1,01-02-62-2008-A-00159,1982-06-05,Male,2017-02-06,P,No signs (patient screened and TB not suspected),No ARV,NaN,0B,7,2.0
2,02-01-62-2015-A-00359,1987-05-26,Female,2015-07-01,P,No signs (patient screened and TB not suspected),No ARV,NaN,0B,7,2.0
3,02-01-62-2015-A-00359,1987-05-26,Female,2015-07-29,P,No signs (patient screened and TB not suspected),No ARV,NaN,0B,7,2.0
4,02-01-62-2015-A-00360,2010-07-14,Male,2015-07-01,P,No signs (patient screened and TB not suspected),No ARV,NaN,0B,7,2.0


In [21]:
a = fac['regioncode']
b = fac['districtcode']
c = fac['facilitycode']

In [22]:
a.value_counts().index
b.value_counts().index
c.value_counts().index

Index(['0B'], dtype='object')

In [23]:
s = a[0]
ss = int(s)
t = b[0]
u = c[0]

In [24]:
facility1 = facii[(facii['ProvinceCode'] == ss) & (facii['DistrictCode'] == t) & (facii['FacilityCode'] == u)]

In [25]:
facility2 = facility1[['ProvinceCode', 'DistrictCode', 'FacilityCode', 'FacilityName']]

In [26]:
facility3 = pd.merge(facility2, reg.rename(columns={'RegionCode':'ProvinceCode'}), on=['ProvinceCode'])

In [27]:
facility4 = pd.merge(facility3, dist.rename(columns={'RegionCode':'ProvinceCode'}), on=['ProvinceCode','DistrictCode'])

In [28]:
facility=pd.read_sql("select tblconfig.regioncode,tblconfig.districtcode,tblconfig.facilitycode,tblsetupregions.Region,tblsetupdistricts.District,facility.FacilityName from tblconfig JOIN tblsetupregions ON tblconfig.regioncode=tblsetupregions.RegionCode JOIN tblsetupdistricts ON tblconfig.districtcode=tblsetupdistricts.DistrictCode AND tblsetupregions.RegionCode=tblsetupdistricts.RegionCode JOIN facility ON tblconfig.facilitycode = facility.FacilityCode AND tblconfig.districtcode=facility.DistrictID AND tblsetupdistricts.DistrictCode = facility.DistrictID AND facility.ProvinceID=tblsetupregions.RegionCode",con=mydb)

In [29]:
facility5 = facility4.rename(columns={'ProvinceCode': 'regioncode', 'DistrictCode': 'districtcode', 'FacilityCode': 'facilitycode'})

In [30]:
facility = facility5[['regioncode','districtcode','facilitycode','Region','District','FacilityName']]

In [31]:
facility.head()

,regioncode,districtcode,facilitycode,Region,District,FacilityName
0,2,7,0B,Mashonaland Central,Shamva,Madziwa Rural Hospital


In [32]:
## Facility
data_v['FacilityName']=facility['FacilityName']
faility_name=data_v['FacilityName']
faility_name[1:n] = faility_name[0]
data_v['FacilityName']=faility_name
filename= faility_name[0]
## District
data_v['District']=facility['District']
district_name=data_v['District']
district_name[1:n] = district_name[0]
data_v['District']=district_name
## Province
data_v['Province']=facility['Region']
province_name=data_v['Province']
province_name[1:n] = province_name[0]
data_v['Province']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [33]:
data_v['Date'] = pd.to_datetime(data_v['VisitDate'],errors='coerce')

In [34]:
data_v['Period'] = pd.DatetimeIndex(data_v['Date']).month

In [35]:
del data_v['Date']

In [36]:
today = datetime.today()

In [37]:
data_v['Age'] = data_v['DateOfBirth'].apply(
               lambda x: today.year - x.year - 
               ((today.month, today.day) < (x.month, x.day)) 
               )

In [38]:
data_v = data_v[['PatientID','DateOfBirth','Age','Sex','VisitDate','Period','VisitTypeCode','TBStatusDescription','ARVStatusDescription','Facilitycode','Districtcode','Provincedoce','FacilityName','District','Province']]

In [39]:
data_v.head()

,PatientID,DateOfBirth,Age,Sex,VisitDate,Period,VisitTypeCode,TBStatusDescription,ARVStatusDescription,Facilitycode,Districtcode,Provincedoce,FacilityName,District,Province
0,01-02-62-2008-A-00159,1982-06-05,39,Male,2015-09-11,9,P,No signs (patient screened and TB not suspected),No ARV,0B,7,2.0,Madziwa Rural Hospital,Shamva,Mashonaland Central
1,01-02-62-2008-A-00159,1982-06-05,39,Male,2017-02-06,2,P,No signs (patient screened and TB not suspected),No ARV,0B,7,2.0,Madziwa Rural Hospital,Shamva,Mashonaland Central
2,02-01-62-2015-A-00359,1987-05-26,34,Female,2015-07-01,7,P,No signs (patient screened and TB not suspected),No ARV,0B,7,2.0,Madziwa Rural Hospital,Shamva,Mashonaland Central
3,02-01-62-2015-A-00359,1987-05-26,34,Female,2015-07-29,7,P,No signs (patient screened and TB not suspected),No ARV,0B,7,2.0,Madziwa Rural Hospital,Shamva,Mashonaland Central
4,02-01-62-2015-A-00360,2010-07-14,11,Male,2015-07-01,7,P,No signs (patient screened and TB not suspected),No ARV,0B,7,2.0,Madziwa Rural Hospital,Shamva,Mashonaland Central


In [40]:
s = facility['FacilityName'].str[:45].value_counts().index

In [41]:
b=s[0]

In [42]:
data_v.to_csv('/home/ignatious/Documents/tb screening/%s.csv'%b, encoding='utf-8', index=False)

# RETENTION

In [43]:
path = "/home/ignatious/Documents/Retention/%s/"%b

In [44]:
os.makedirs(path, exist_ok=True) 

In [45]:
df_visits = pd.read_sql('SELECT * FROM tblvisits', con=mydb)
df_patients = pd.read_sql("Select * from tblpatients",con=mydb)
df_patients_prior_exp = pd.read_sql("Select * from tblpatientspriorexposure",con=mydb)
df_pregnacies = pd.read_sql("Select * from tblpregnancies ",con=mydb)
df_tests = pd.read_sql("Select * from tbltests",con=mydb)
df_status = pd.read_sql("Select * from tblstatus",con=mydb)
df_problems = pd.read_sql("Select * from tblpatientsproblems",con=mydb)
df_appointments = pd.read_sql("Select * from tblappointments",con=mydb)
df_medication = pd.read_sql("Select * from tblmedication",con=mydb)
df_exposed_infants = pd.read_sql("Select * from tblexposedinfants",con=mydb)
df_exposed_infants_tests = pd.read_sql("Select * from tblexposedinfanttests",con=mydb)
df_exposed_infants_visits = pd.read_sql("Select * from tblexposedinfantvisits",con=mydb)
df_exposed_infants_medication = pd.read_sql("Select * from tblexposedinfantmedication",con=mydb)

In [46]:
fac=pd.read_sql("select c.regioncode,c.districtcode,c.facilitycode from tblconfig c",con=mydb)

fac['regioncode'] = fac.regioncode.astype(int)
fac['districtcode'] = fac.districtcode.astype(str)
fac['facilitycode'] = fac.facilitycode.astype(str)

In [47]:
faility_name=fac['facilitycode']
district_name=fac['districtcode']
province_name=fac['regioncode']
facility=str(faility_name[0])
district=str(district_name[0])
province = str(province_name[0])
province="0"+province
Facility_code=province+district+facility

In [48]:
## Facility
n=df_visits.shape[0]
df_visits['Facilitycode']=fac['facilitycode']
faility_name=df_visits['Facilitycode']
faility_name[1:n] = faility_name[0]
df_visits['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_visits['Districtcode']=fac['districtcode']
district_name=df_visits['Districtcode']
district_name[1:n] = district_name[0]
df_visits['Districtcode']=district_name
## Province
df_visits['Provincecode']=fac['regioncode']
province_name=df_visits['Provincecode']
province_name[1:n] = province_name[0]
df_visits['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap

In [49]:
n=df_patients.shape[0]
df_patients['Facilitycode']=fac['facilitycode']
faility_name=df_patients['Facilitycode']
faility_name[1:n] = faility_name[0]
df_patients['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_patients['Districtcode']=fac['districtcode']
district_name=df_patients['Districtcode']
district_name[1:n] = district_name[0]
df_patients['Districtcode']=district_name
## Province
df_patients['Provincecode']=fac['regioncode']
province_name=df_patients['Provincecode']
province_name[1:n] = province_name[0]
df_patients['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [50]:
n=df_patients_prior_exp.shape[0]
df_patients_prior_exp['Facilitycode']=fac['facilitycode']
faility_name=df_patients_prior_exp['Facilitycode']
faility_name[1:n] = faility_name[0]
df_patients_prior_exp['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_patients_prior_exp['Districtcode']=fac['districtcode']
district_name=df_patients_prior_exp['Districtcode']
district_name[1:n] = district_name[0]
df_patients_prior_exp['Districtcode']=district_name
## Province
df_patients_prior_exp['Provincecode']=fac['regioncode']
province_name=df_patients_prior_exp['Provincecode']
province_name[1:n] = province_name[0]
df_patients_prior_exp['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [51]:
n=df_pregnacies.shape[0]
df_pregnacies['Facilitycode']=fac['facilitycode']
faility_name=df_pregnacies['Facilitycode']
faility_name[1:n] = faility_name[0]
df_pregnacies['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_pregnacies['Districtcode']=fac['districtcode']
district_name=df_pregnacies['Districtcode']
district_name[1:n] = district_name[0]
df_pregnacies['Districtcode']=district_name
## Province
df_pregnacies['Provincecode']=fac['regioncode']
province_name=df_pregnacies['Provincecode']
province_name[1:n] = province_name[0]
df_pregnacies['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [52]:
n=df_tests.shape[0]
df_tests['Facilitycode']=fac['facilitycode']
faility_name=df_tests['Facilitycode']
faility_name[1:n] = faility_name[0]
df_tests['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_tests['Districtcode']=fac['districtcode']
district_name=df_tests['Districtcode']
district_name[1:n] = district_name[0]
df_tests['Districtcode']=district_name
## Province
df_tests['Provincecode']=fac['regioncode']
province_name=df_tests['Provincecode']
province_name[1:n] = province_name[0]
df_tests['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [53]:
n=df_status.shape[0]
df_status['Facilitycode']=fac['facilitycode']
faility_name=df_status['Facilitycode']
faility_name[1:n] = faility_name[0]
df_status['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_status['Districtcode']=fac['districtcode']
district_name=df_status['Districtcode']
district_name[1:n] = district_name[0]
df_status['Districtcode']=district_name
## Province
df_status['Provincecode']=fac['regioncode']
province_name=df_status['Provincecode']
province_name[1:n] = province_name[0]
df_status['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [54]:
n=df_problems.shape[0]
df_problems['Facilitycode']=fac['facilitycode']
faility_name=df_problems['Facilitycode']
faility_name[1:n] = faility_name[0]
df_problems['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_problems['Districtcode']=fac['districtcode']
district_name=df_problems['Districtcode']
district_name[1:n] = district_name[0]
df_problems['Districtcode']=district_name
## Province
df_problems['Provincecode']=fac['regioncode']
province_name=df_problems['Provincecode']
province_name[1:n] = province_name[0]
df_problems['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [55]:
n=df_appointments.shape[0]
df_appointments['Facilitycode']=fac['facilitycode']
faility_name=df_appointments['Facilitycode']
faility_name[1:n] = faility_name[0]
df_appointments['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_appointments['Districtcode']=fac['districtcode']
district_name=df_appointments['Districtcode']
district_name[1:n] = district_name[0]
df_appointments['Districtcode']=district_name
## Province
df_appointments['Provincecode']=fac['regioncode']
province_name=df_appointments['Provincecode']
province_name[1:n] = province_name[0]
df_appointments['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [56]:
n=df_medication.shape[0]
df_medication['Facilitycode']=fac['facilitycode']
faility_name=df_medication['Facilitycode']
faility_name[1:n] = faility_name[0]
df_medication['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_medication['Districtcode']=fac['districtcode']
district_name=df_medication['Districtcode']
district_name[1:n] = district_name[0]
df_medication['Districtcode']=district_name
## Province
df_medication['Provincecode']=fac['regioncode']
province_name=df_medication['Provincecode']
province_name[1:n] = province_name[0]
df_medication['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [57]:
n=df_exposed_infants.shape[0]
df_exposed_infants['Facilitycode']=fac['facilitycode']
faility_name=df_exposed_infants['Facilitycode']
faility_name[1:n] = faility_name[0]
df_exposed_infants['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_exposed_infants['Districtcode']=fac['districtcode']
district_name=df_exposed_infants['Districtcode']
district_name[1:n] = district_name[0]
df_exposed_infants['Districtcode']=district_name
## Province
df_exposed_infants['Provincecode']=fac['regioncode']
province_name=df_exposed_infants['Provincecode']
province_name[1:n] = province_name[0]
df_exposed_infants['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [58]:
n=df_exposed_infants_tests.shape[0]
df_exposed_infants_tests['Facilitycode']=fac['facilitycode']
faility_name=df_exposed_infants_tests['Facilitycode']
faility_name[1:n] = faility_name[0]
df_exposed_infants_tests['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_exposed_infants_tests['Districtcode']=fac['districtcode']
district_name=df_exposed_infants_tests['Districtcode']
district_name[1:n] = district_name[0]
df_exposed_infants_tests['Districtcode']=district_name
## Province
df_exposed_infants_tests['Provincecode']=fac['regioncode']
province_name=df_exposed_infants_tests['Provincecode']
province_name[1:n] = province_name[0]
df_exposed_infants_tests['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [59]:
n=df_exposed_infants_visits.shape[0]
df_exposed_infants_visits['Facilitycode']=fac['facilitycode']
faility_name=df_exposed_infants_visits['Facilitycode']
faility_name[1:n] = faility_name[0]
df_exposed_infants_visits['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_exposed_infants_visits['Districtcode']=fac['districtcode']
district_name=df_exposed_infants_visits['Districtcode']
district_name[1:n] = district_name[0]
df_exposed_infants_visits['Districtcode']=district_name
## Province
df_exposed_infants_visits['Provincecode']=fac['regioncode']
province_name=df_exposed_infants_visits['Provincecode']
province_name[1:n] = province_name[0]
df_exposed_infants_visits['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [60]:
n=df_exposed_infants_medication.shape[0]
df_exposed_infants_medication['Facilitycode']=fac['facilitycode']
faility_name=df_exposed_infants_medication['Facilitycode']
faility_name[1:n] = faility_name[0]
df_exposed_infants_medication['Facilitycode']=faility_name
filename= faility_name[0]
## District
df_exposed_infants_medication['Districtcode']=fac['districtcode']
district_name=df_exposed_infants_medication['Districtcode']
district_name[1:n] = district_name[0]
df_exposed_infants_medication['Districtcode']=district_name
## Province
df_exposed_infants_medication['Provincecode']=fac['regioncode']
province_name=df_exposed_infants_medication['Provincecode']
province_name[1:n] = province_name[0]
df_exposed_infants_medication['Provincecode']=province_name

/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ignatious/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [61]:
df_visits = df_visits.applymap(str)
df_patients = df_patients.applymap(str)
df_patients_prior_exp = df_patients_prior_exp.applymap(str)
df_pregnacies = df_pregnacies.applymap(str)
df_tests = df_tests.applymap(str)
df_status = df_status.applymap(str)
df_problems = df_problems.applymap(str)
df_appointments = df_appointments.applymap(str)
df_medication = df_medication.applymap(str)
df_exposed_infants = df_exposed_infants.applymap(str)
df_exposed_infants_tests = df_exposed_infants_tests.applymap(str)
df_exposed_infants_visits = df_exposed_infants_visits.applymap(str)
df_exposed_infants_medication = df_exposed_infants_medication.applymap(str)

In [62]:
ext = ".dta"

In [63]:
visit_path =path+'tblvisits'+ext
prio_exp_path =path+'tblpatientspriorexposure'+ext
preg_path=path+'tblpregnancies'+ext
pat_parth=path+'tblpatients'+ext
test_path=path+'tbltests'+ext
status_path=path+'tblstatus'+ext
problems_path=path+'tblpatientsproblems'+ext
app_path=path+'tblappointments'+ext
med_path=path+'tblmedication'+ext
exp_inf_path=path+'tblexposedinfants'+ext
exp_inf_test_path=path+'tblexposedinfanttests'+ext
exp_inf_visits_path=path+'tblexposedinfantvisits'+ext
exp_inf_med_path=path+'tblexposedinfantmedication'+ext

In [64]:
export_visits = df_visits.to_stata(visit_path, version=118, convert_strl = df_visits.columns[df_visits.isnull().all() & df_visits.dtypes==object].tolist())
export_prio_expos= df_patients_prior_exp.to_stata(prio_exp_path, version=118, convert_strl = df_patients_prior_exp.columns[df_patients_prior_exp.isnull().all() & df_patients_prior_exp.dtypes==object].tolist())
export_pregs = df_pregnacies.to_stata(preg_path, version=118, convert_strl = df_pregnacies.columns[df_pregnacies.isnull().all() & df_pregnacies.dtypes==object].tolist())
export_pats= df_patients.to_stata(pat_parth, version=118, convert_strl = df_patients.columns[df_patients.isnull().all() & df_patients.dtypes==object].tolist())
export_tests= df_tests.to_stata(test_path, version=118, convert_strl = df_tests.columns[df_tests.isnull().all() & df_tests.dtypes==object].tolist())
export_stutaus= df_status.to_stata(status_path, version=118, convert_strl = df_status.columns[df_status.isnull().all() & df_status.dtypes==object].tolist())
export_problems= df_problems.to_stata(problems_path, version=118, convert_strl = df_problems.columns[df_problems.isnull().all() & df_problems.dtypes==object].tolist())
export_apps= df_appointments.to_stata(app_path, version=118, convert_strl = df_appointments.columns[df_appointments.isnull().all() & df_appointments.dtypes==object].tolist())
export_meds= df_medication.to_stata(med_path, version=118, convert_strl = df_medication.columns[df_medication.isnull().all() & df_medication.dtypes==object].tolist())
export_exp_infs= df_exposed_infants.to_stata(exp_inf_path, version=118, convert_strl = df_exposed_infants.columns[df_exposed_infants.isnull().all() & df_exposed_infants.dtypes==object].tolist())
export_exp_infs_tests= df_exposed_infants_tests.to_stata(exp_inf_test_path, version=118, convert_strl = df_exposed_infants_tests.columns[df_exposed_infants_tests.isnull().all() & df_exposed_infants_tests.dtypes==object].tolist())
export_exp_infs_visits= df_exposed_infants_visits.to_stata(exp_inf_visits_path, version=118, convert_strl = df_exposed_infants_visits.columns[df_exposed_infants_visits.isnull().all() & df_exposed_infants_visits.dtypes==object].tolist())
export_exp_infs_meds= df_exposed_infants_medication.to_stata(exp_inf_med_path, version=118, convert_strl = df_exposed_infants_medication.columns[df_exposed_infants_medication.isnull().all() & df_exposed_infants_medication.dtypes==object].tolist())

In [65]:
mydb.close()